# 패키지 불러오기

In [1]:
'''
본 버전은 FA선수에 대해서만 적용하였음
1. 종속변수 할당 지정
2. 물가지수 반영(CPI)
3. 1차FA기간, 2차FA기간, 3차FA기간 등 변수생성
4. FA여부, 1차FA기간, 2차FA기간, 3차FA기간에 대해서 1칸씩 SHIFT
'''

import pandas as pd
import numpy as np

# 전처리 전체 Flow

### 1. 타자
#### 1. 기본데이터 전처리
 - "생년월일" --> 총 4개(년, 월, 일, 나이) 파생변수 생성
 - 기존의 "생년월일" 변수는 삭제
 - "볼넷/삼진", "땅볼/뜬공" --> 변수제거
 - "연봉" 변수 내 ','를 제거
 
#### 2. 물가상승률을 반영하여 연봉 변화
 - 한국은행(CPI)기반 2020년으로 연봉가치 통일

#### 3. 종속변수 할당
 - ex) 2016년도 성적을 2017년도 연봉이랑 매칭
 - "ID", "선수명" 변수는 제거
 
#### 4. train/test 데이터 분할
 - test: 2020년 데이터
 - train: 그 외 데이터 모두
 - train/test 분할 후 각각에 대해 결측치가 1개라도 있는 행은 제거
 - "팀명"과 "포지션(수비)" 원핫인코딩 (실시한 것과 안한 것 모두 생성)

### 2. 투수
#### 1. 기본데이터 전처리
 - "생년월일" --> 총 4개(년, 월, 일, 나이) 파생변수 생성
 - 기존의 "생년월일" 변수는 삭제
 - "이닝수" 0인 선수는 제거
 - "승률" --> 0승 0패는 0으로 대체
 - "삼진/볼넷", "땅볼/뜬공" --> 변수제거
 - "포지션(수비)" 변수는 삭제
 - "연봉" 변수 내 ','를 제거

#### 2. 물가상승률을 반영하여 연봉 변화
 - 한국은행(CPI)기반 2020년으로 연봉가치 통일

#### 3. 종속변수 할당
 - ex) 2016년도 성적을 2017년도 연봉이랑 매칭
 - "ID", "선수명" 변수는 제거
 
#### 4. train/test 데이터 분할
 - test: 2020년 데이터
 - train: 그 외 데이터 모두
 - train/test 분할 후 각각에 대해 결측치가 1개라도 있는 행은 제거
 - "팀명" 원핫인코딩 (실시한 것과 안한 것 모두 생성)

In [2]:
# ID와 선수명변수 제거 및 결측행 제거
hitter = pd.read_csv('선수데이터(전처리완료)/타자.csv')

# 생년월일 전처리
hitter['출생연도'] = hitter['생년월일'].apply(lambda x: x.split('-')[0]).astype('int')
hitter['출생월'] = hitter['생년월일'].apply(lambda x: x.split('-')[1]).astype('int')
hitter['출생일'] = hitter['생년월일'].apply(lambda x: x.split('-')[2]).astype('int')
hitter['나이'] = hitter['연도'] - hitter['출생연도'] + 1
hitter = hitter.drop('생년월일', axis=1)

# 데뷔년도 데이터 형변환
hitter['데뷔년도'] = hitter['데뷔년도'].astype('int')

In [3]:
# 타자데이터 전처리
### 공격데이터 형변환
hitter['타율'] = hitter['타율'].replace('-', 0).astype('float')
hitter['장타율'] = hitter['장타율'].replace('-', 0).astype('float')
hitter['순수장타율'] = hitter['순수장타율'].replace('-', 0).astype('float')
hitter = hitter.drop(['볼넷/삼진', '땅볼/뜬공'], axis=1)

# 수비데이터 전처리
### 수비데이터 형변환
hitter[['경기(수비)', '선발경기(수비)', '실책(수비)', '견제사(수비)', '자살(수비)', '보살(수비)', '병살(수비)', '포일(수비)', '도루허용(수비)', '도루실패(수비)']] = hitter[['경기(수비)', '선발경기(수비)', '실책(수비)', '견제사(수비)', '자살(수비)', '보살(수비)', '병살(수비)', '포일(수비)', '도루허용(수비)', '도루실패(수비)']].astype('int')

In [4]:
# 연봉 및 수상횟수 전처리

def salary_preprocessing(value):
    try:
        value = value.replace(',', '')
    
    except:
        pass
    
    return value

hitter['연봉'] = list(map(lambda x: salary_preprocessing(x), hitter['연봉']))
hitter['연봉'] = hitter['연봉'].astype('float')

hitter['수상횟수'] = hitter['수상횟수'].astype('int')

### 2. 투수
#### 1. 기본데이터 전처리
 - "ID", "선수명" 변수는 제거
 - 결측치가 하나라도 있는 행은 제거

 - 생년월일 --> 년/월/일 변수 생성
 - 생년월일 --> 연도데이터를 기반으로 나이변수 생성
 - 기존의 "생년월일" 변수는 삭제
 - 팀명데이터 --> 팀별로 원핫인코딩
 - 기존의 "팀명" 변수 삭제

    
#### 2. 투수데이터 전처리
 - 이닝수가 0인 행은 제거 --> **논의필요**
 - 0승 0패의 승률: 0으로 처리  --> **논의필요**
 - 땅볼/뜬공, 삼진/볼넷 --> 변수제거 --> **논의필요**
 
#### 3. 수비데이터 전처리
 - 기존의 "포지션(수비)" 변수 삭제 --> 수비의 수비포지션은 only 수비
   
#### 4. 논의필요
 - 승률 --> 0승0패인경우 -로 존재
 - 평균자책점, 이닝당_출루허용률, 피안타율, 투구수/이닝, 9이닝당 삼진, 9이닝당 볼넷, 삼진/볼넷, 피장타율 --> 이닝이 0인 경우 때문에 -로존재
 - 땅볼/뜬공, 삼진/볼넷 --> 타자와 비슷한 이슈

In [5]:
# ID와 선수명변수 제거 및 결측행 제거
pitcher = pd.read_csv('선수데이터(전처리완료)/투수.csv')

# 생년월일 전처리
pitcher['출생연도'] = pitcher['생년월일'].apply(lambda x: x.split('-')[0]).astype('int')
pitcher['출생월'] = pitcher['생년월일'].apply(lambda x: x.split('-')[1]).astype('int')
pitcher['출생일'] = pitcher['생년월일'].apply(lambda x: x.split('-')[2]).astype('int')
pitcher['나이'] = pitcher['연도'] - pitcher['출생연도'] + 1
pitcher = pitcher.drop('생년월일', axis=1)

# 데뷔년도 데이터 형변환
pitcher['데뷔년도'] = pitcher['데뷔년도'].astype('int')

# 이닝수가 1/3이상인 선수만 분석
pitcher = pitcher[pitcher['이닝'] != 0].reset_index(drop=True)
pitcher

,ID,선수명,데뷔년도,팀명,연도,평균자책점,경기,승리,패배,세이브,...,FA여부,뉴스개수(연도별정규화),뉴스개수(누적정규화),수상횟수,누적이닝,포지션 엔트로피,출생연도,출생월,출생일,나이
0,63492,이재익,2013,삼성,2020,135.0,4,0,0,0,...,0,0.001455,0.001455,0.0,0.666667,0.0,1994,3,18,27
1,67588,윤성빈,2017,롯데,2018,6.39,18,2,5,0,...,0,0.131754,0.131754,0.0,50.666667,0.0,1999,2,26,20
2,67588,윤성빈,2017,롯데,2019,81.0,1,0,1,0,...,0,0.053388,0.185142,0.0,51.000000,0.0,1999,2,26,21
3,77829,김광현,2007,SK,2007,3.62,20,3,7,0,...,0,0.481259,0.481259,0.0,77.000000,0.0,1988,7,22,20
4,77829,김광현,2007,SK,2008,2.39,27,16,4,0,...,0,1.000000,1.481259,2.0,239.000000,0.0,1988,7,22,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3520,96251,박명환,1996,LG,2008,8.61,5,0,3,0,...,0,0.058981,1.916030,0.0,858.000000,0.0,1977,6,7,32
3521,96251,박명환,1996,LG,2009,6.19,4,0,1,0,...,0,0.066025,1.982055,0.0,874.000000,0.0,1977,6,7,33
3522,96251,박명환,1996,LG,2010,6.63,15,4,6,0,...,0,0.054240,2.036295,0.0,950.000000,0.0,1977,6,7,34
3523,96251,박명환,1996,NC,2014,7.2,5,0,2,0,...,0,0.031619,2.067914,0.0,955.000000,0.0,1977,6,7,38


In [6]:
# 투구데이터 전처리
pitcher['승률'] = pitcher['승률'].replace('-',0).astype('float')
pitcher[['평균자책점', '이닝당_출루허용률', '피안타율', '투구수/이닝', '9이닝당 삼진', '9이닝당 볼넷', '피장타율']] = pitcher[['평균자책점', '이닝당_출루허용률', '피안타율', '투구수/이닝', '9이닝당 삼진', '9이닝당 볼넷', '피장타율']].astype('float')
pitcher = pitcher.drop(['삼진/볼넷', '땅볼/뜬공'], axis=1)

# 수비데이터 전처리
### 포지션(수비) 변수 삭제
pitcher = pitcher.drop('포지션(수비)', axis=1)

In [7]:
# 연봉 및 수상횟수 전처리
pitcher['연봉'] = list(map(lambda x: salary_preprocessing(x), pitcher['연봉']))
pitcher['연봉'] = pitcher['연봉'].astype('float')

pitcher['수상횟수'] = pitcher['수상횟수'].astype('int')

# 2. 물가상승률(CPI)반영

#### (1) 물가상승률 계산

In [8]:
money_rate = pd.read_excel('물가상승률데이터/소비자_물가지수(한국은행).xlsx')
money_rate = money_rate.loc[:, '2002':].transpose()
money_rate = money_rate.reset_index()
money_rate.columns = ['연도','소비자물가지수']

for idx in range(money_rate.shape[0]):
    money_rate.loc[idx, '2020년기준상수'] = money_rate.loc[money_rate.shape[0]-1, '소비자물가지수'] / money_rate.loc[idx, '소비자물가지수']
    
money_rate

c:\users\sungs\appdata\local\programs\python\python38\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,연도,소비자물가지수,2020년기준상수
0,2002,71.193,1.480764
1,2003,73.695,1.430491
2,2004,76.341,1.380909
3,2005,78.444,1.343889
4,2006,80.202,1.314431
5,2007,82.235,1.281936
6,2008,86.079,1.224689
7,2009,88.452,1.191833
8,2010,91.051,1.157813
9,2011,94.717,1.113000


#### (2) 연봉 변환

In [9]:
for i in range(hitter.shape[0]):
    year = hitter.loc[i, '연도']
    interest_rate = float(money_rate.loc[money_rate['연도'] == f'{year}', '2020년기준상수'])
    hitter.loc[i, '연봉'] = hitter.loc[i, '연봉'] * interest_rate


for i in range(pitcher.shape[0]):
    year = pitcher.loc[i, '연도']
    interest_rate = float(money_rate.loc[money_rate['연도'] == f'{year}', '2020년기준상수'])
    pitcher.loc[i, '연봉'] = pitcher.loc[i, '연봉'] * interest_rate

# 3. 종속변수 할당

#### (1) 타자

In [10]:
hitter_final = pd.DataFrame()

for ID in hitter['ID'].unique():

    tmp = hitter.loc[hitter['ID'] == ID].reset_index(drop=True)

    for i in range(tmp.shape[0]):
        current_year = tmp.loc[i,'연도']

        try:
            money = tmp.loc[(tmp['연도'] == (current_year+1)), '연봉'].values[0]
        except:
            money = np.nan

        tmp.loc[i, '수정연봉'] = money

    tmp.drop('연봉', axis=1, inplace=True)
    tmp = tmp.rename(columns = {'수정연봉':'연봉'})

    hitter_final = pd.concat([hitter_final, tmp]).reset_index(drop=True)

#### (2) 투수

In [11]:
pitcher_final = pd.DataFrame()

for ID in pitcher['ID'].unique():

    tmp = pitcher.loc[pitcher['ID'] == ID].reset_index(drop=True)

    for i in range(tmp.shape[0]):
        current_year = tmp.loc[i,'연도']

        try:
            money = tmp.loc[(tmp['연도'] == (current_year+1)), '연봉'].values[0]
        except:
            money = np.nan

        tmp.loc[i, '수정연봉'] = money

    tmp.drop('연봉', axis=1, inplace=True)
    tmp = tmp.rename(columns = {'수정연봉':'연봉'})

    pitcher_final = pd.concat([pitcher_final, tmp]).reset_index(drop=True)

# 4. FA연도별 변수 생성
 - 선수명, 연도기준 MERGE (3가지가 모두 겹치는 경우가 없는 것을 식별 완료)
  - (이때, 타자 김태균만 동명이인+동일연도가 존재하기에 이에 대해서만 전처리 실시)

In [12]:
fa_data = pd.read_csv('FA정보데이터/FA_정보.csv')
fa_data = fa_data[~((fa_data['선수명'] == '이승호') & (fa_data['생년월일'] == '1976-08-23'))].reset_index(drop=True) # 불필요한 데이터 삭제
fa_data = fa_data[['선수명','연도','계약기간']]
fa_data = fa_data.dropna().reset_index(drop=True)
fa_data['계약기간'] = fa_data['계약기간'].apply(lambda x: int(x.replace('년','')))
fa_data                  

,선수명,연도,계약기간
0,오주원,2020,2
1,김태균,2020,1
2,오재원,2020,3
3,김태군,2020,4
4,이성열,2020,2
...,...,...,...
225,안경현,2003,4
226,양준혁,2002,4
227,김민재,2002,4
228,전준호,2002,3


In [13]:
# 타자 계약기간 data merge 
fa_ids = hitter_final.loc[hitter_final['FA여부'] == 1, 'ID'].unique()
hitter_final_fa = hitter_final[hitter_final['ID'].isin(fa_ids)].sort_values(by=['ID','연도']).reset_index(drop=True)
hitter_final_fa = pd.merge(hitter_final_fa, fa_data, how='left', on=['선수명','연도'])

# merge 예외사항 후처리
hitter_final_fa.loc[(hitter_final_fa['ID'] == 71752) & (hitter_final_fa['연도'] == 2005), '계약기간'] = np.nan

In [14]:
# 투수 계약기간 data merge 
fa_ids = pitcher_final.loc[pitcher_final['FA여부'] == 1, 'ID'].unique()
pitcher_final_fa = pitcher_final[pitcher_final['ID'].isin(fa_ids)].sort_values(by=['ID','연도']).reset_index(drop=True)
pitcher_final_fa = pd.merge(pitcher_final_fa, fa_data, how='left', on=['선수명','연도'])

In [15]:
# 타자: 1차, 2차, 3차 파생변수 생성
hitter_final_fa2 = pd.DataFrame()
for ID in hitter_final_fa['ID'].unique():
    
    tmp = hitter_final_fa[hitter_final_fa['ID'] == ID].reset_index(drop=True)
    years = list(tmp.loc[tmp['FA여부'] == 1, '연도'])

    for idx, year in enumerate(years):

        contract_years = [year+i for i in range(int(tmp.loc[tmp['연도'] == year, '계약기간']))]

        if idx == 0:
            tmp.loc[tmp['연도'].isin(contract_years), '1차FA여부'] = 1
        elif idx == 1:
            tmp.loc[tmp['연도'].isin(contract_years), '2차FA여부'] = 1
        elif idx == 2:
            tmp.loc[tmp['연도'].isin(contract_years), '3차FA여부'] = 1 
        else:
            print(f'{ID}에 4차 이상의 FA존재')
            
    hitter_final_fa2 = pd.concat([hitter_final_fa2, tmp]).reset_index(drop=True)

In [16]:
# 투수: 1차, 2차, 3차 파생변수 생성
pitcher_final_fa2 = pd.DataFrame()
for ID in pitcher_final_fa['ID'].unique():
    
    tmp = pitcher_final_fa[pitcher_final_fa['ID'] == ID].reset_index(drop=True)
    years = list(tmp.loc[tmp['FA여부'] == 1, '연도'])

    for idx, year in enumerate(years):

        contract_years = [year+i for i in range(int(tmp.loc[tmp['연도'] == year, '계약기간']))]

        if idx == 0:
            tmp.loc[tmp['연도'].isin(contract_years), '1차FA여부'] = 1
        elif idx == 1:
            tmp.loc[tmp['연도'].isin(contract_years), '2차FA여부'] = 1
        elif idx == 2:
            tmp.loc[tmp['연도'].isin(contract_years), '3차FA여부'] = 1 
        else:
            print(f'{ID}에 4차 이상의 FA존재')
            
    pitcher_final_fa2 = pd.concat([pitcher_final_fa2, tmp]).reset_index(drop=True)

In [17]:
# 계약기간 변수 삭제
hitter_final_fa2 = hitter_final_fa2.drop('계약기간', axis=1)
pitcher_final_fa2 = pitcher_final_fa2.drop('계약기간', axis=1)

# 파생변수 null값에 0으로 채우기
hitter_final_fa2['1차FA여부'] = hitter_final_fa2['1차FA여부'].fillna(0)
hitter_final_fa2['2차FA여부'] = hitter_final_fa2['2차FA여부'].fillna(0)
hitter_final_fa2['3차FA여부'] = hitter_final_fa2['3차FA여부'].fillna(0)

pitcher_final_fa2['1차FA여부'] = pitcher_final_fa2['1차FA여부'].fillna(0)
pitcher_final_fa2['2차FA여부'] = pitcher_final_fa2['2차FA여부'].fillna(0)

# 5. FA여부, 1차/2차/3차 FA여부 1칸씩 shift

In [18]:
pitcher_final_fa3 = pd.DataFrame()

for ID in pitcher_final_fa2['ID'].unique():
    
    tmp = pitcher_final_fa2[pitcher_final_fa2['ID'] == ID].reset_index(drop=True)
    
    tmp['FA여부'] = tmp['FA여부'].shift(-1)
    tmp['1차FA여부'] = tmp['1차FA여부'].shift(-1)
    tmp['2차FA여부'] = tmp['2차FA여부'].shift(-1)
    
    pitcher_final_fa3 = pd.concat([pitcher_final_fa3, tmp]).reset_index(drop=True)

In [19]:
hitter_final_fa3 = pd.DataFrame()

for ID in hitter_final_fa2['ID'].unique():
    
    tmp = hitter_final_fa2[hitter_final_fa2['ID'] == ID].reset_index(drop=True)
    
    tmp['FA여부'] = tmp['FA여부'].shift(-1)
    tmp['1차FA여부'] = tmp['1차FA여부'].shift(-1)
    tmp['2차FA여부'] = tmp['2차FA여부'].shift(-1)
    tmp['3차FA여부'] = tmp['3차FA여부'].shift(-1)
    
    hitter_final_fa3 = pd.concat([hitter_final_fa3, tmp]).reset_index(drop=True)

# 6. 원핫인코딩 후 train, test 분할
 - 타자: 팀명, 포지션(수비)
 - 투수: 팀명

#### (1) 원핫인코딩

In [20]:
# 팀명데이터 원핫인코딩
hitter_final_onehot = pd.get_dummies(hitter_final_fa3, columns=['팀명'], prefix='팀명', drop_first=False, dtype='int')
hitter_final_onehot = pd.get_dummies(hitter_final_onehot, columns=['포지션(수비)'], prefix='포지션(수비)', drop_first=False, dtype='int')

# 2020년만 test, 그 외는 train
hitter_onehot_train = hitter_final_onehot[hitter_final_onehot['연도'] != 2020].dropna().reset_index(drop=True)
hitter_onehot_test = hitter_final_onehot[hitter_final_onehot['연도'] == 2020].reset_index(drop=True)

hitter_onehot_train.shape, hitter_onehot_test.shape

((1220, 91), (41, 91))

In [21]:
# 팀명데이터 원핫인코딩
pitcher_final_onehot = pd.get_dummies(pitcher_final_fa3, columns=['팀명'], prefix='팀명', drop_first=False, dtype='int')

# 2020년만 test, 그 외는 train
pitcher_onehot_train = pitcher_final_onehot[pitcher_final_onehot['연도'] != 2020].dropna().reset_index(drop=True)
pitcher_onehot_test = pitcher_final_onehot[pitcher_final_onehot['연도'] == 2020].reset_index(drop=True)

pitcher_onehot_train.shape, pitcher_onehot_test.shape

((644, 85), (19, 85))

#### (2) 원핫인코딩 미실시

In [22]:
# 2020년만 test, 그 외는 train
hitter_train = hitter_final_fa3[hitter_final_fa3['연도'] != 2020].dropna().reset_index(drop=True)
hitter_test = hitter_final_fa3[hitter_final_fa3['연도'] == 2020].reset_index(drop=True)

hitter_train.shape, hitter_test.shape

((1220, 73), (41, 73))

In [23]:
# 2020년만 test, 그 외는 train
pitcher_train = pitcher_final_fa3[pitcher_final_fa3['연도'] != 2020].dropna().reset_index(drop=True)
pitcher_test = pitcher_final_fa3[pitcher_final_fa3['연도'] == 2020].reset_index(drop=True)

pitcher_train.shape, pitcher_test.shape

((644, 75), (19, 75))

# 저장

In [24]:
hitter_onehot_train.to_csv('선수데이터(전처리완료)/모델링용ver4/타자(모델링용_원핫인코딩)_train_ver4.csv', index=False, encoding='utf-8-sig')
hitter_onehot_test.to_csv('선수데이터(전처리완료)/모델링용ver4/타자(모델링용_원핫인코딩)_test_ver4.csv', index=False, encoding='utf-8-sig')
hitter_train.to_csv('선수데이터(전처리완료)/모델링용ver4/타자(모델링용)_train_ver4.csv', index=False, encoding='utf-8-sig')
hitter_test.to_csv('선수데이터(전처리완료)/모델링용ver4/타자(모델링용)_test_ver4.csv', index=False, encoding='utf-8-sig')

pitcher_onehot_train.to_csv('선수데이터(전처리완료)/모델링용ver4/투수(모델링용_원핫인코딩)_train_ver4.csv', index=False, encoding='utf-8-sig')
pitcher_onehot_test.to_csv('선수데이터(전처리완료)/모델링용ver4/투수(모델링용_원핫인코딩)_test_ver4.csv', index=False, encoding='utf-8-sig')
pitcher_train.to_csv('선수데이터(전처리완료)/모델링용ver4/투수(모델링용)_train_ver4.csv', index=False, encoding='utf-8-sig')
pitcher_test.to_csv('선수데이터(전처리완료)/모델링용ver4/투수(모델링용)_test_ver4.csv', index=False, encoding='utf-8-sig')